Завдання 1

In [3]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357281 sha256=927a6eff7d557d39c6b8d465fd1a124ab7c0f684ab5a8d336c68d065ab5a7ae1
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [4]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
#1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.
data = Dataset.load_builtin('ml-100k')

for row in data.raw_ratings[:5]:
    print(row)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [6]:
#2. Виведіть перші 5 рядків завантаженого датасету.
for row in data.raw_ratings[:5]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів")

Користувач 196 оцінив фільм 242 на 3.0 балів
Користувач 186 оцінив фільм 302 на 3.0 балів
Користувач 22 оцінив фільм 377 на 1.0 балів
Користувач 244 оцінив фільм 51 на 2.0 балів
Користувач 166 оцінив фільм 346 на 1.0 балів


In [7]:
#3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.
trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [8]:
#4. Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.
#5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки(MAE).
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

#5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки(MAE).
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 0.7260298744094941
Найкращий MAE для KNNBasic: 0.8032761283455914
Вибраний алгоритм: SVD


In [9]:
#6. Виведи рекомендації (10 фільмів) для конкретного користувача.
best_model.fit(trainset)

user_id = str(153)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 153: 18
Топ-10 фільмів, рекомендованих для користувача:
Фільм 1449 з прогнозованим рейтингом 4.03
Фільм 603 з прогнозованим рейтингом 3.92
Фільм 513 з прогнозованим рейтингом 3.89
Фільм 169 з прогнозованим рейтингом 3.82
Фільм 641 з прогнозованим рейтингом 3.78
Фільм 316 з прогнозованим рейтингом 3.76
Фільм 475 з прогнозованим рейтингом 3.71
Фільм 48 з прогнозованим рейтингом 3.70
Фільм 615 з прогнозованим рейтингом 3.69
Фільм 427 з прогнозованим рейтингом 3.69


Висновок. Завантажили датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise. Реалізуйвали два алгоритми для рекомендаційної системи на основі цього
датасету. Використали крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів (найкращий MAE для SVD: 0.7260298744094941, найкращий MAE для KNNBasic: 0.8032761283455914). Обрали найкращий алгоритм на основі середньої абсолютної помилки(Вибраний алгоритм: SVD). Вивели рекомендації (10 фільмів) для конкретного користувача.

Завдання 2

In [12]:
#1. Побудуйте власну рекомендаційну систему (приклад є в документації). Оцініть її.
model = SVD()
model.fit(trainset)
predictions = model.test(testset)

accuracy.rmse(predictions)

def get_top_n_for_user(predictions, uid, n=10):
    user_ratings = []

    for prediction in predictions:
        if prediction.uid == uid:
            user_ratings.append((prediction.iid, prediction.est))
    user_ratings.sort(key=lambda x: x[1], reverse=True)

    return user_ratings[:n]
#2. Отримайте рекомендацію для певного користувача.
specific_user_id = '33'

top_n_recommendations_for_user = get_top_n_for_user(predictions, specific_user_id, n=10)

print(f"Рекомендації для користувача {specific_user_id}:")
for (iid, rating) in top_n_recommendations_for_user:
    print(f"Фільм {iid} з передбаченим рейтингом {rating:.2f}")

RMSE: 0.9434
Рекомендації для користувача 33:
Фільм 313 з передбаченим рейтингом 4.32
Фільм 307 з передбаченим рейтингом 3.79
Фільм 328 з передбаченим рейтингом 3.72
Фільм 872 з передбаченим рейтингом 3.60
Фільм 348 з передбаченим рейтингом 3.57
Фільм 895 з передбаченим рейтингом 3.47
Фільм 245 з передбаченим рейтингом 3.44
Фільм 682 з передбаченим рейтингом 3.40
Фільм 323 з передбаченим рейтингом 3.20
Фільм 260 з передбаченим рейтингом 2.80
